In [1]:
# Packages, seed and path
## packages
from edge_sim_py import *
from GNN import MyGNN
from torch_geometric.data import Data
import torch
from CNNDQN import DQNAgent
import numpy as np
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt

## seed
import torch
torch.manual_seed(5)
import random
random.seed(5)
import numpy as np
np.random.seed(5)

## path
algo_name = "gcn_lm"

/home/mascalmeida/miniconda3/envs/edgeai/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Functions
# def custom_collect_method(self) -> dict: # Custom collect method to measure the power consumption of each server
#     metrics = {
#         "Instance ID": self.id,
#         "Power Consumption": self.get_power_consumption(),
#     }
#     return metrics

global network_links_adj_list #Our graph of adjacent network switches
global network_switch_dict #dict of network switches to edge servers
global edge_server_dict #dict of edge servers to network switches
global matrix_list
global network_switch_index #network switch index mapping
global agent
global optimizer
global model

criterion = nn.MSELoss() #Mean Squared error criterion provided by pytorch
reward_list = list()
power_list = list() # List to store total power consumption everytime the task scheduling algorithm is used

def my_algorithm(parameters):
  
    print("\n\n")
    total_reward = 0
    total_power = 0 #We sum the power consumption after migrating each service
    for service in Service.all(): #Iterate over every service
        

        
        #Create a list of our EdgeServer states
        state_vector = list()
        next_state_vector = list()
        
        
        #Initialise them with zeros
        for _ in range(len(NetworkSwitch.all())):
            state_vector.append(np.zeros(4))
            next_state_vector.append(np.zeros(4))

        state_vector = np.array(state_vector)
        next_state_vector = np.array(next_state_vector)

        if not service.being_provisioned:
            
            #We treat each edge server connected to the same network sitch as one node, and hence, sum their current utilizations
            for edge_server in EdgeServer.all():
                edge_server_cpu = edge_server.cpu
                edge_server_memory = edge_server.memory
                edge_server_disk = edge_server.disk
                power = (edge_server_cpu * edge_server_memory * edge_server_disk) ** (1 / 3)
                vector = [edge_server_cpu, edge_server_memory, edge_server_disk, power]
                vector = np.array(vector)
                state_vector[network_switch_index[edge_server_dict[edge_server]]] = np.add(state_vector[network_switch_index[edge_server_dict[edge_server]]],vector)

            
            
            #print(state_vector)

            matrix_array = np.array(matrix_list)

            #print(matrix_array)

            #Create state and adjacency list tensors

            state_vector_tensor  = torch.tensor(state_vector,dtype=torch.float64)
            adjacency_list_tensor = torch.tensor(matrix_array,dtype=torch.int64)

            state_vector_tensor = state_vector_tensor.to(model.lin1.weight.dtype)

            
            #create our dataset and pass to model
            data = Data(x=state_vector_tensor, edge_index=adjacency_list_tensor)
            output = model(data.x, data.edge_index)

#            output = output.unsqueeze(0).unsqueeze(0)


            #pass graph network output to Q network
            output = output.detach()
            action = agent.choose_action(output)

            #To conserve resources, we don't want to migrate back to our host 
            if(service.server == EdgeServer.all()[action]):
                 break

            print(f"[STEP {parameters['current_step']}] Migrating {service} From {service.server} to {EdgeServer.all()[action]}")

            #Migrate service to new edgeserver
            service.provision(target_server=EdgeServer.all()[action])

            reward = 0
            power = 0

            #Get our next state, after taking action

            for edge_server in EdgeServer.all():
                edge_server_cpu = edge_server.cpu
                edge_server_memory = edge_server.memory
                edge_server_disk = edge_server.disk
                power = (edge_server_cpu * edge_server_memory * edge_server_disk) ** (1 / 3)
                vector = [edge_server_cpu, edge_server_memory, edge_server_disk, power]
                next_state_vector[network_switch_index[edge_server_dict[edge_server]]] = np.add(next_state_vector[network_switch_index[edge_server_dict[edge_server]]],vector)
                reward = reward + 1/edge_server.get_power_consumption() #Our reward is the inverse of the edge server's power consumption
                power = power + edge_server.get_power_consumption() #get the sum of powerconsumption of each edge server


            
            #Get our next state output from the graphical neural network
            next_state_vector_tensor  = torch.tensor(next_state_vector,dtype=torch.float64)
            adjacency_list_tensor = torch.tensor(matrix_array,dtype=torch.int64)

            next_state_vector_tensor = next_state_vector_tensor.to(model.lin1.weight.dtype)

            data = Data(x=next_state_vector_tensor, edge_index=adjacency_list_tensor)
            next_output = model(data.x, data.edge_index)

#            next_output = next_output.unsqueeze(0).unsqueeze(0)

            next_output = next_output.detach()
            
            #print(output.shape)
            #print(next_output.shape)
            
            
            #Use the next state to update the Deep Q network
            agent.update(output,action,next_output,reward,False)


            
            #Retrieve our Q network loss, and use it to update our GNN
            loss = agent.loss

            #print(loss)

            loss = loss.clone().detach()

            loss = criterion(torch.tensor(loss),torch.tensor(0))

            loss = torch.tensor(loss, requires_grad=True)


  #          print(loss)

#            loss = torch.tensor(loss,dtype=torch.float64,requires_grad=True)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_reward += reward
            total_power+=power #Sum our power consumption

    reward_list.append(total_reward)
    power_list.append(total_power) #Append power consumption to power list for plotting
    agent.epsilon*=agent.epsilon_decay #Reduce the probability of agent taking random action for exploration

def stopping_criterion(model: object):    
    # As EdgeSimPy will halt the simulation whenever this function returns True,
    # its output will be a boolean expression that checks if the current time step is 600
    return model.schedule.steps == 1000

In [3]:
simulator = Simulator(
    tick_duration=1,
    tick_unit="seconds",
    stopping_criterion=stopping_criterion,
    resource_management_algorithm=my_algorithm,
)

# Loading a sample dataset
#simulator.initialize(input_file="sample_dataset3.json")
simulator.initialize(input_file="https://raw.githubusercontent.com/EdgeSimPy/edgesimpy-tutorials/master/datasets/sample_dataset2.json")

#Assigning the custom collect method
#EdgeServer.collect = custom_collect_method

network_links_adj_list = {} #Our graph of adjacent network switches
network_switch_index = {} #network switch index mapping
matrix_list = list()
matrix_list.append(list())
matrix_list.append(list())

i = 0

for network_switch in NetworkSwitch.all(): #Initialise with empty list for each network switch, and assign index
    network_links_adj_list[network_switch] = list()
    network_switch_index[network_switch] = i
    i += 1

for network_link in NetworkLink.all(): #Iterate through network links and add the connected switches to our adjacency list
    print(network_link.nodes)
    network_links_adj_list[network_link.nodes[0]].append(network_link.nodes[1])
    network_links_adj_list[network_link.nodes[1]].append(network_link.nodes[0])
    matrix_list[0].append(network_switch_index[network_link.nodes[1]])
    matrix_list[1].append(network_switch_index[network_link.nodes[0]])

#dicts containing network switch to edge server mappings

network_switch_dict = {}
edge_server_dict = {}

#Map our edge serers and network switches

for network_switch in NetworkSwitch.all():
    print(network_switch.edge_servers)
    
    if(len(network_switch.edge_servers)):
        network_switch_dict[network_switch] = network_switch.edge_servers
        
        for edge_server in network_switch.edge_servers:
            edge_server_dict[edge_server] = network_switch

#Initialise our DQN agent and out CNN agent
agent = DQNAgent(4, len(EdgeServer.all()))
model = MyGNN(in_channels=4, out_channels=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Executing the simulation
simulator.run_model()

[NetworkSwitch_1, NetworkSwitch_5]
[NetworkSwitch_1, NetworkSwitch_2]
[NetworkSwitch_2, NetworkSwitch_5]
[NetworkSwitch_2, NetworkSwitch_6]
[NetworkSwitch_2, NetworkSwitch_3]
[NetworkSwitch_3, NetworkSwitch_6]
[NetworkSwitch_3, NetworkSwitch_7]
[NetworkSwitch_3, NetworkSwitch_4]
[NetworkSwitch_4, NetworkSwitch_7]
[NetworkSwitch_4, NetworkSwitch_8]
[NetworkSwitch_5, NetworkSwitch_9]
[NetworkSwitch_5, NetworkSwitch_10]
[NetworkSwitch_5, NetworkSwitch_6]
[NetworkSwitch_6, NetworkSwitch_10]
[NetworkSwitch_6, NetworkSwitch_11]
[NetworkSwitch_6, NetworkSwitch_7]
[NetworkSwitch_7, NetworkSwitch_11]
[NetworkSwitch_7, NetworkSwitch_12]
[NetworkSwitch_7, NetworkSwitch_8]
[NetworkSwitch_8, NetworkSwitch_12]
[NetworkSwitch_9, NetworkSwitch_13]
[NetworkSwitch_9, NetworkSwitch_10]
[NetworkSwitch_10, NetworkSwitch_13]
[NetworkSwitch_10, NetworkSwitch_14]
[NetworkSwitch_10, NetworkSwitch_11]
[NetworkSwitch_11, NetworkSwitch_14]
[NetworkSwitch_11, NetworkSwitch_15]
[NetworkSwitch_11, NetworkSwitch_12]


/home/mascalmeida/miniconda3/envs/edgeai/lib/python3.7/site-packages/ipykernel_launcher.py:138: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/mascalmeida/miniconda3/envs/edgeai/lib/python3.7/site-packages/ipykernel_launcher.py:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).








[STEP 24] Migrating Service_1 From EdgeServer_1 to EdgeServer_6






[STEP 26] Migrating Service_1 From EdgeServer_6 to EdgeServer_2



[STEP 27] Migrating Service_1 From EdgeServer_2 to EdgeServer_1
[STEP 27] Migrating Service_2 From EdgeServer_1 to EdgeServer_6
[STEP 27] Migrating Service_3 From EdgeServer_2 to EdgeServer_6
[STEP 27] Migrating Service_4 From EdgeServer_1 to EdgeServer_2
[STEP 27] Migrating Service_5 From EdgeServer_6 to EdgeServer_2
[STEP 27] Migrating Service_6 From EdgeServer_2 to EdgeServer_6



[STEP 28] Migrating Service_1 From EdgeServer_1 to EdgeServer_2



[STEP 29] Migrating Service_1 From EdgeServer_2 to EdgeServer_1
[STEP 29] Migrating Service_2 From EdgeServer_6 to EdgeServer_2
[STEP 29] Migrating Service_4 From EdgeServer_2 to EdgeServer_6
[STEP 29] Migrating Service_5 From EdgeServer_2 to EdgeServer_6



[STEP 30] Migrating Service_1 From EdgeServer_1 to EdgeServer_6
[STEP 30] Migrating Service_2 From EdgeServer_2 to EdgeServer_6
[STEP 30] Migrat

In [4]:
# Results
## Retrieving logs dataframe for plot
logs_containerregistry = pd.DataFrame(simulator.agent_metrics["ContainerRegistry"])
logs_edgeserver = pd.DataFrame(simulator.agent_metrics["EdgeServer"])
logs_edgeserver['CPU Usage'] = (logs_edgeserver['CPU Demand']*100)/logs_edgeserver['CPU']
logs_edgeserver['RAM Usage'] = (logs_edgeserver['RAM Demand']*100)/logs_edgeserver['RAM']
logs_networkflow = pd.DataFrame(simulator.agent_metrics["NetworkFlow"])
logs_networkswitch = pd.DataFrame(simulator.agent_metrics["NetworkSwitch"])
logs_service = pd.DataFrame(simulator.agent_metrics["Service"])
logs_user = pd.DataFrame(simulator.agent_metrics["User"])

dt_edge = pd.concat([
    logs_edgeserver[['Object', 'Power Consumption']].groupby(by=['Object']).sum().reset_index(),
    logs_edgeserver[['Object', 'CPU Usage', 'RAM Usage']].groupby(by=['Object']).mean().reset_index(drop=True)
    ], axis=1)
dt_edge_final = pd.DataFrame(dt_edge.mean()).reset_index().rename(columns={'index':'parameters', 0:'values'})
display(dt_edge, dt_edge_final)

/home/mascalmeida/miniconda3/envs/edgeai/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


,Object,Power Consumption,CPU Usage,RAM Usage
0,EdgeServer_1,196915.113500,31.031469,31.031469
1,EdgeServer_2,167015.905500,0.861638,0.861638
2,EdgeServer_3,67133.895700,0.049950,0.099900
3,EdgeServer_4,67247.152150,0.124875,0.249750
4,EdgeServer_5,74776.783333,8.474858,6.462288
5,EdgeServer_6,102419.175000,28.646354,42.969530


,parameters,values
0,Power Consumption,112584.670864
1,CPU Usage,11.531524
2,RAM Usage,13.612429
